# 处理数据集

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
tokenizer = AutoTokenizer.from_pretrained('qwen/Qwen2-7B-Instruct')

/home/zwb/.cache/kgenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-23 19:16:52.723577: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-23 19:16:52.775248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 19:16:54.540677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Te

In [2]:
df = pd.read_json('./dataset/tianji-wishes-chinese-v0.1.json')
ds = Dataset.from_pandas(df)
df = pd.DataFrame(ds)
df = df.explode('conversation')  # 展平 'conversation' 列
df = pd.json_normalize(df['conversation'])  # 将嵌套的 JSON 展平
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['input', 'output', 'system'],
    num_rows: 2976
})

In [22]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [23]:
tokenized_id = dataset.map(process_func, remove_columns=dataset.column_names)
tokenized_id

Map: 100%|██████████| 2976/2976 [00:01<00:00, 2565.69 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2976
})

In [25]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福<|im_end|>\n<|im_start|>user\n我想送赵老师生日祝福,严肃风格<|im_end|>\n<|im_start|>assistant\n尊敬的赵老师，值此生辰之际，愿岁月如诗，为您带来无尽的喜悦与美好；愿时光荏苒，为您留下珍贵的回忆与感悟。愿您快快乐乐，事业更上一层楼，教诲之恩桃李满天下。在这充满敬意的时刻，恭祝赵老师生日快乐，幸福安康！<|endoftext|>'

In [27]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'赵老师，值此春节佳节之际，恭祝您福寿安康，万事如意。在过去的一年里，您的辛勤耕耘为后辈树立了榜样，新春到来，愿您的生活如诗如画，工作更上一层楼，继续以您的智慧和热忱，引领我们前行。岁月静好，愿您享受每一个温馨时刻，幸福安康，喜悦无忧。<|endoftext|>'

# 创建模型

In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

In [29]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [30]:
model.dtype

torch.bfloat16

# lora 

In [31]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'gate_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [32]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'gate_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [33]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [34]:
args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=10, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [35]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

# 合并加载模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

lora_path = './output/Qwen2_instruct_lora/checkpoint-550' 

tokenizer = AutoTokenizer.from_pretrained('qwen/Qwen2-7B-Instruct', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

# 模型推理

In [4]:
prompt = "我想祝福郑同学中秋节快乐,小红书风格"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

郑同学，中秋佳节至，月圆人团圆🌕🎉，愿你的学业如月般圆满，心情如饼般甜蜜！月饼香，思念长，快乐伴你行哦～
